In [ ]:
# Install dependicies
!pip install textblob
!pip install tweepy
!pip install wordcloud
!pip install kafka-python
!pip install nltk
!pip install wordcloud


In [ ]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
import json
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

kafka_server_ip = "172.17.249.145"


# Offline tweets


In [ ]:
from kafka import KafkaConsumer
import json, time

topic_name = 'twitterstreams'
tweets_batch_size = 100

def percentage(part,whole):
    return 100 * float(part)/float(whole) 

class SentiConsumer():
    
    def __init__(self):
        # create consumer config
        self.consumer = KafkaConsumer(topic_name,
                                      bootstrap_servers=[f'{kafka_server_ip}:9092'],
                                      auto_offset_reset='earliest',
                                      enable_auto_commit=True,
                                      auto_commit_interval_ms =  5000,
                                      fetch_max_bytes = 128,
                                      max_poll_records = 100,
                                     )
        
    def create_wordcloud(self, text):
        
        #mask = np.array(Image.open("cloud.png"))
        stopwords = set(STOPWORDS)
        wc = WordCloud(background_color="white",
                       #mask = mask,
                       max_words=3000,
                       stopwords=stopwords,
                       width = 1000, height = 200,
                       repeat=True)
        wc.generate(str(text))
        wc.to_file("wc.png")
        print("Word Cloud Saved Successfully")
        path="wc.png"
        display(Image.open(path))

    def sentiment_analysis(self, tweets):
        
        positive  = 0
        negative = 0
        neutral = 0
        polarity = 0
        tweet_list = []
        neutral_list = []
        negative_list = []
        positive_list = []

        for tweet in tweets:

            #print(tweet.text)
            tweet_list.append(tweet)
            analysis = TextBlob(tweet)
            score = SentimentIntensityAnalyzer().polarity_scores(tweet)
            neg = score['neg']
            neu = score['neu']
            pos = score['pos']
            comp = score['compound']
            polarity += analysis.sentiment.polarity

            if neg > pos:
                negative_list.append(tweet)
                negative += 1

            elif pos > neg:
                positive_list.append(tweet)
                positive += 1

            elif pos == neg:
                neutral_list.append(tweet)
                neutral += 1

        positive = percentage(positive, tweets_batch_size)
        negative = percentage(negative, tweets_batch_size)
        neutral = percentage(neutral, tweets_batch_size)
        polarity = percentage(polarity, tweets_batch_size)
        positive = format(positive, '.1f')
        negative = format(negative, '.1f')
        neutral = format(neutral, '.1f')
        
        #Number of Tweets (Total, Positive, Negative, Neutral)
        tweet_list = pd.DataFrame(tweet_list)
        neutral_list = pd.DataFrame(neutral_list)
        negative_list = pd.DataFrame(negative_list)
        positive_list = pd.DataFrame(positive_list)
        print("total number: ",len(tweet_list))
        print("positive number: ",len(positive_list))
        print("negative number: ", len(negative_list))
        print("neutral number: ",len(neutral_list))
        
        print("Total no of tweets processed {}".format(len(tweets)))
        
        #Creating PieCart
        labels = ['Positive ['+str(positive)+'%]' , 'Neutral ['+str(neutral)+'%]','Negative ['+str(negative)+'%]']
        sizes = [positive, neutral, negative]
        colors = ['yellowgreen', 'blue','red']
        patches, texts = plt.pie(sizes,colors=colors, startangle=90)
        plt.style.use('default')
        plt.legend(labels)
        plt.title("Sentiment Analysis Result")
        plt.axis('equal')
        plt.show()
        
        # create word cloud
        print("Word cloud for positive words")
        self.create_wordcloud(positive_list)
        print("Word cloud for negative words")
        self.create_wordcloud(negative_list)
        print("Word cloud for neutral words")
        self.create_wordcloud(neutral_list)
    
    def process(self):
        
        batch_messages = []
        batch_size = 0
        
        for message in self.consumer:
            tweet = message.value
            batch_messages.append(str(tweet))
            batch_size += 1
            
            if batch_size>=tweets_batch_size:
                # perform sentiment analysis on batch messages
                print(f"New iteration........................................................... {batch_size} {tweets_batch_size}")
                print(batch_messages[-1])
                self.sentiment_analysis(batch_messages)
                batch_size = 0 # reset batch
                batch_messages = []
                

In [ ]:
senti_consumer = SentiConsumer()

In [ ]:
senti_consumer.process()

In [ ]:
senti_consumer.consumer.close()